In [2]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load("it_core_news_lg")

In [3]:
df = pd.read_excel("phonItalia 1.10 - word forms.xlsx")
df

,nLem,fqTot,fqTotL,fqQuo,fqPer,fqLib,dispT,dispQ,dispP,dispL,...,OLD,OLDF,OLD_RelFreq,PLD,PLDF,PLD_RelFreq,BG_Sum,BG_Mean,BP_Sum,BP_Mean
0,3560,50518,10.830085,23982,17679,8857,0.99,0.99,0.99,0.99,...,1.00,6.004370,1.8037,1.00,6.758153,1.602521,0,0.000000,0,0.000000
1,3560,5219,8.560061,2391,1948,880,0.99,0.99,0.98,0.99,...,1.00,6.004370,1.425639,1.00,6.758153,1.266627,0,0.000000,0,0.000000
2,3560,1544,7.342132,537,625,382,0.97,0.99,0.99,0.97,...,1.00,6.004370,1.222798,1.00,6.758153,1.086411,0,0.000000,0,0.000000
3,3560,272,5.605802,112,80,80,0.94,0.96,0.90,0.95,...,1.00,6.004370,0.93362,1.00,6.758153,0.829487,0,0.000000,0,0.000000
4,3562,234,5.455321,109,107,18,0.84,0.76,0.79,0.79,...,1.00,6.004370,0.908558,1.00,6.758153,0.807221,0,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,89330,5,1.609438,5,0,0,0.00,0.00,0.00,0.00,...,1.85,1.790223,0.899015,1.90,1.499083,1.073615,196047,49011.750000,135076,33769.000000
119996,89331,2,0.693147,2,0,0,0.00,0.00,0.00,0.00,...,2.45,0.910700,0.761115,1.90,1.499083,0.462381,205893,41178.601563,135076,33769.000000
119997,89332,1,0.000000,1,0,0,0.00,0.00,0.00,0.00,...,2.40,1.129599,0,2.35,1.007284,0,509593,84932.164063,432115,72019.164063
119998,89334,1,0.000000,0,1,0,0.00,0.00,0.00,0.00,...,5.50,1.142923,0,5.70,0.947322,0,1068756,97159.632813,969221,88111.000000


In [4]:
df = df[['fqTot', 'gramCat', 'lemma', 'word', 'Phones']] # only keep relevant columns
df = df[df['fqTot'] > 1] # only keep words with freq > 1
df

,fqTot,gramCat,lemma,word,Phones
0,50518,P,a,a,a
1,5219,P IN P@,a,a,a
2,1544,P IN B@,a,a,a
3,272,P IN C@,a,a,a
4,234,S,a,a,a
...,...,...,...,...,...
119992,28,E,zurigo,zurigo,ZuriGo
119994,2,E,zurletti,zurletti,Zurletti
119995,5,E,zurli,zurli,Zurli
119996,2,E IN E@,zurli',zurli',Zurli


In [55]:
df = pd.read_excel("ita_deriv.xlsx")

In [5]:
# gcs = dict(zip(zip(df.index, df["word"]), df["gramCat"]))
import collections
gcs = collections.defaultdict(list)

for idx, row in tqdm.tqdm(df.iterrows()):
    gcs[row["word"]].append(row["gramCat"])

73823it [00:02, 26584.21it/s]


In [54]:
spacify = {
    'B': 'ADV',
    'E': 'NOUN',
    'G': 'ADJ',
    'V': 'VERB',
    'I': 'INTJ',
    'N': 'PRON',
    'S': 'NOUN'
}

from spacy.language import Language

# seen = {}
accs = collections.defaultdict(int)


@Language.component("gtagger")
def custom_tagger(doc):
    print("calling ct", doc)
#     print("analyzing", doc, id(doc))
#     input()
    # Modify POS tags or other annotations as needed
    for token in doc:
        # Modify token attributes as required
        t = doc.text
        i = id(doc)
#         if i in seen:
#             token.pos_ = seen[i]
        
        if gc := gcs.get(t):
            try:
                token.pos_ = spacify.get(gc[accs[t]], 'X')
                token.tag_ = ''
                accs[t] += 1
                print(t, accs[t], gc, token.pos_)
#                 seen[i] = token.pos_
            except:
                print(t, accs[t])
        else:
            print(token.text)
            input()
            token.pos_ = 'X'
#         break

    print([token.pos_ for token in doc])
    return doc

nlp = spacy.load("it_core_news_lg", exclude=['tagger', 'morphologizer'])
# nlp.initialize()
print(nlp.pipeline)

# config = {"overwrite": False}
nlp.add_pipe("morphologizer") #, overwrite=False)

nlp.add_pipe('gtagger', name='gtagger', before='morphologizer')
nlp.initialize()

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x215432500>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x1e7575460>), ('lemmatizer', <spacy.pipeline.edit_tree_lemmatizer.EditTreeLemmatizer object at 0x215431fc0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x215440040>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x1e75762d0>)]


ValueError: [E143] Labels for component 'morphologizer' not initialized. This can be fixed by calling add_label, or by providing a representative batch of examples to the component's `initialize` method.

In [53]:
# nlp.morphologizer

AttributeError: 'Italian' object has no attribute 'morphologizer'

In [57]:
import tqdm
nlp = spacy.load("it_core_news_lg", exclude=['tagger'])
df['morph'] = ''
df['pos'] = ''

for col in ['Gender', 'Number', 'Mood', 'Person', 'Tense', 'VerbForm', 'Clitic', 'PronType', 'Foreign', 'Degree', 'NumType', 'Definite', 'Poss']:
    df[col] = ''

for idx, row in tqdm.tqdm(df.iterrows()):
    try:
        doc = nlp(row['word'])
        tok = doc[0]
        df.loc[idx, 'morph'] = str(tok.morph)
        df.loc[idx, 'pos'] = tok.pos_
#         print()
#         print(tok, tok.morph, tok.pos_)
#         input()
        for k, v in tok.morph.to_dict().items():
            df.loc[idx, k] = v
    except Exception as e:
        print(e, idx, row)


73823it [04:40, 263.53it/s]

[E1041] Expected a string, Doc, or bytes as input, but got: <class 'float'> 73810 Unnamed: 0.4           73810
Unnamed: 0.3           73810
Unnamed: 0.2             NaN
Unnamed: 0.1             NaN
Unnamed: 0               NaN
fqTot                    NaN
gramCat                  NaN
lemma                    NaN
word                     NaN
Phones                   NaN
morph                       
pos                         
Gender                      
Number                      
Mood                        
Person                      
Tense                       
VerbForm                    
Clitic                      
PronType                    
Foreign                     
Degree                      
NumType                     
Definite                    
Poss                        
actual_lemma             NaN
actual_lemma_pos         NaN
extra                    NaN
actual_lemma_phones      NaN
spacy_lemma              NaN
nltk_lemma               NaN
derivational       

In [13]:
keys.keys()

dict_keys(['Gender', 'Number', 'Mood', 'Person', 'Tense', 'VerbForm', 'Clitic', 'PronType', 'Foreign', 'Degree', 'NumType', 'Definite', 'Poss'])

In [ ]:
df['morph'].value_counts()

In [32]:
df.loc[5, 'pos']

'ADP'

In [58]:
df.to_excel("ita_deriv.xlsx")

In [43]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1e96dc460>),
 ('lemmatizer',
  <spacy.pipeline.edit_tree_lemmatizer.EditTreeLemmatizer at 0x1e96df580>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1ee9c23c0>)]